# 07 — Pandas Advanced Manipulation & Reshaping (1DownLabs)

Goals:
- build instinct for pandas patterns (groupby/transform/window/reshape)
- solve realistic analytics tasks without googling every step
- learn the mental models: split-apply-combine + tidy data

Dataset: vgsales.csv

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

project_root = Path("..").resolve()
sys.path.append(str(project_root))

df = pd.read_csv(project_root / "data" / "raw" / "vgsales.csv")

df.columns = (df.columns.str.strip().str.lower().str.replace(" ", "_"))
df = df.dropna(subset=["year"]).copy()
df["year"] = df["year"].astype(int)

df.head()

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Pandas weapons (master these and 80% becomes easy)

1) `groupby().agg(...)`  → summarise
2) `groupby().transform(...)` → compute per-row group metrics (same length as df)
3) `merge(...)` → bring metrics back
4) `pivot_table(...)` / `melt(...)` → reshape wide/long
5) Sorting + ranking (`sort_values`, `rank`) → top-n within groups

In [3]:
# 3) transform vs agg (Code)
# Goal: add publisher_total_sales to every row (same length as df)
df["publisher_total_sales"] = df.groupby("publisher")["global_sales"].transform("sum")
df[["name", "publisher", "global_sales", "publisher_total_sales"]].head(10)

,name,publisher,global_sales,publisher_total_sales
0,Wii Sports,Nintendo,82.74,1784.43
1,Super Mario Bros.,Nintendo,40.24,1784.43
2,Mario Kart Wii,Nintendo,35.82,1784.43
3,Wii Sports Resort,Nintendo,33.00,1784.43
4,Pokemon Red/Pokemon Blue,Nintendo,31.37,1784.43
5,Tetris,Nintendo,30.26,1784.43
6,New Super Mario Bros.,Nintendo,30.01,1784.43
7,Wii Play,Nintendo,29.02,1784.43
8,New Super Mario Bros. Wii,Nintendo,28.62,1784.43
9,Duck Hunt,Nintendo,28.31,1784.43


In [4]:
# Now: share of each game within publisher
df["share_within_publisher"] = df["global_sales"] / df["publisher_total_sales"]
df[["name", "publisher", "global_sales", "share_within_publisher"]].sort_values("share_within_publisher", ascending=False).head(10)

,name,publisher,global_sales,share_within_publisher
10534,DJ Max Fever,PM Studios,0.10,1.0
14889,Daito Giken Koushiki Pachi-Slot Simulator Hiho...,Paon Corporation,0.03,1.0
11385,Kamen Rider: Travelers Senki,Seventh Chord,0.08,1.0
15082,Prison Architect,Introversion Software,0.02,1.0
15075,Shaman King: Chou Senjiryokketsu 3,King Records,0.02,1.0
9447,Diablo II,Havas Interactive,0.13,1.0
2516,Custer's Revenge,Mystique,0.82,1.0
7726,Shutter Island,Merscom LLC,0.20,1.0
13005,Air Traffic Chaos,Ertain,0.05,1.0
10697,Fate/hollow ataraxia,Type-Moon,0.10,1.0


In [5]:
# Top-N within groups (publisher’s top 3 games) (Code)

df_sorted = df.sort_values(["publisher", "global_sales"], ascending=[True, False]).copy()
df_sorted["pub_rank"] = df_sorted.groupby("publisher")["global_sales"].rank(method="first", ascending=False)

top3_each_pub = df_sorted[df_sorted["pub_rank"] <= 3][["publisher", "name", "platform", "year", "global_sales", "pub_rank"]]
top3_each_pub.head(20)

,publisher,name,platform,year,global_sales,pub_rank
12349,10TACLE Studios,Panzer Tactics,DS,2007,0.06,1.0
14130,10TACLE Studios,Boulder Dash: Rocks!,DS,2007,0.03,2.0
15706,10TACLE Studios,Pirates: Legend of the Black Buccaneer,PS2,2006,0.02,3.0
13057,1C Company,Men of War: Assault Squad,PC,2011,0.05,1.0
13724,1C Company,Off-Road Drive,PC,2011,0.04,2.0
16449,1C Company,King's Bounty: Armored Princess,PC,2009,0.01,3.0
2596,20th Century Fox Video Games,Alien,2600,1981,0.79,1.0
5389,20th Century Fox Video Games,Fantastic Voyage,2600,1981,0.34,2.0
5395,20th Century Fox Video Games,Bank Heist,2600,1982,0.34,3.0
14011,2D Boy,World of Goo,PC,2008,0.04,1.0


In [6]:
'''Window-style: cumulative sums by year (Code)
Total global sales per year + cumulative'''

year_sales = df.groupby("year", as_index=False)["global_sales"].sum().sort_values("year")
year_sales["cumulative_global_sales"] = year_sales["global_sales"].cumsum()
year_sales.head()

,year,global_sales,cumulative_global_sales
0,1980,11.38,11.38
1,1981,35.77,47.15
2,1982,28.86,76.01
3,1983,16.79,92.80
4,1984,50.36,143.16


In [7]:
# Rolling metrics (3-year rolling mean) (Code)

year_sales["rolling_3yr_avg"] = year_sales["global_sales"].rolling(window=3, min_periods=1).mean()
year_sales.tail()

,year,global_sales,cumulative_global_sales,rolling_3yr_avg
34,2014,337.05,8484.65,356.233333
35,2015,264.44,8749.09,323.200000
36,2016,70.93,8820.02,224.140000
37,2017,0.05,8820.07,111.806667
38,2020,0.29,8820.36,23.756667


In [8]:
# Reshaping: wide ↔ long (pivot and melt) (Code)

# Wide: regional totals per year

regional_by_year = (
    df.groupby("year", as_index=False)
      .agg(na=("na_sales","sum"), eu=("eu_sales","sum"), jp=("jp_sales","sum"), other=("other_sales","sum"))
      .sort_values("year")
)
regional_by_year.head()

,year,na,eu,jp,other
0,1980,10.59,0.67,0.00,0.12
1,1981,33.40,1.96,0.00,0.32
2,1982,26.92,1.65,0.00,0.31
3,1983,7.76,0.80,8.10,0.14
4,1984,33.28,2.10,14.27,0.70


In [9]:
# Long: tidy format for plotting/analysis

regional_long = regional_by_year.melt(id_vars="year", var_name="region", value_name="sales")
regional_long.head()

,year,region,sales
0,1980,na,10.59
1,1981,na,33.40
2,1982,na,26.92
3,1983,na,7.76
4,1984,na,33.28


In [10]:
# Cohort-like thinking: “first appearance year” per platform (Code)

platform_first_year = df.groupby("platform")["year"].min().rename("platform_first_year")
df = df.merge(platform_first_year, on="platform", how="left")
df[["platform","year","platform_first_year"]].head()

,platform,year,platform_first_year
0,Wii,2006,2006
1,NES,1985,1983
2,Wii,2008,2006
3,Wii,2009,2006
4,GB,1996,1988


In [17]:
'''DRILL 1 — “Top Genre per Platform per Decade”

Create a table with columns:

decade (e.g., 1990, 2000, 2010)

platform

top_genre

top_genre_sales'''


df['decade'] = (df['year'] // 10 )*10

top_genre_by_sale = (df.groupby(['platform', 'decade', 'genre']).agg(top_genre_sales = ('global_sales', 'sum')))

top_genre_by_sale['rank'] = top_genre_by_sale.groupby(['platform', 'decade'])['top_genre_sales'].rank(method="first", ascending=False)

top_genre_by_sale_final = top_genre_by_sale[top_genre_by_sale['rank'] == 1]

top_genre_by_sale_final =  top_genre_by_sale_final.reset_index()
top_genre_by_sale_final

,platform,decade,genre,top_genre_sales,rank
0,2600,1980,Action,26.39,1.0
1,3DO,1990,Adventure,0.06,1.0
2,3DS,2010,Role-Playing,75.71,1.0
3,DC,1990,Platform,2.42,1.0
4,DC,2000,Racing,2.01,1.0
5,DS,1980,Action,0.02,1.0
6,DS,2000,Simulation,121.86,1.0
7,DS,2010,Role-Playing,36.84,1.0
8,DS,2020,Simulation,0.29,1.0
9,GB,1980,Puzzle,37.54,1.0


In [20]:
'''DRILL 2 — “Publisher concentration”

For each publisher, compute:

publisher_total_sales

top_game_sales (max global_sales)

concentration = top_game_sales / publisher_total_sales

Return top 20 publishers by concentration (descending), include game name.'''



df['publisher_total_sales'] = df.groupby('publisher')['global_sales'].transform('sum')
df['top_game_sales'] = df.groupby('publisher')['global_sales'].transform('max')
df['concentration'] = (df['top_game_sales'] / df['publisher_total_sales']).round(decimals=2)

top_20_pub_by_concen =  df[['publisher','name','publisher_total_sales','top_game_sales','concentration']].sort_values('concentration', ascending= False).head(20)

top_20_pub_by_concen

,publisher,name,publisher_total_sales,top_game_sales,concentration
16299,UIG Entertainment,Farming 2017 - The Simulation,0.01,0.01,1.0
15307,Technos Japan Corporation,Super Dodge Ball,0.02,0.02,1.0
15254,Karin Entertainment,Zettai Meikyuu Grimm: Nanatsu no Kagi to Rakue...,0.02,0.02,1.0
8121,Adeline Software,Time Commando,0.17,0.17,1.0
7604,Merscom LLC,Shutter Island,0.20,0.20,1.0
16294,Rain Games,Teslagrad,0.01,0.01,1.0
15233,Imageworks,Sotsugyou II: Neo Generation Special,0.02,0.02,1.0
15179,Marvelous Games,Net High,0.02,0.02,1.0
15106,Phoenix Games,Chicken Hunter,0.02,0.02,1.0
15101,Quintet,Code R,0.02,0.02,1.0


In [13]:
'''DRILL 3 — “Year-over-year growth”

Compute total global sales per year and YoY % growth.

Output columns:

year

global_sales

prev_year_sales

yoy_growth_pct'''

yoy_growth = df.groupby('year').agg(total_global_sales = ('global_sales', 'sum')).sort_values('year')

yoy_growth['prev_year_sales'] =  yoy_growth['total_global_sales'].shift(1) 

yoy_growth['yoy_growth_pct'] = (((yoy_growth['total_global_sales'] - yoy_growth['prev_year_sales']) / yoy_growth['prev_year_sales']) * 100.00).round(decimals=2)

yoy_growth

,total_global_sales,prev_year_sales,yoy_growth_pct
year,,,
1980,11.38,NaN,NaN
1981,35.77,11.38,214.32
1982,28.86,35.77,-19.32
1983,16.79,28.86,-41.82
1984,50.36,16.79,199.94
1985,53.94,50.36,7.11
1986,37.07,53.94,-31.28
1987,21.74,37.07,-41.35
1988,47.22,21.74,117.20


In [25]:
'''DRILL 4 — “Stable winners”

Find publishers that were in the top 10 by global sales for at least 5 different years.

Output:

publisher

years_in_top10'''


drill_4 =  df.groupby(['year','publisher'], as_index=False)['global_sales'].sum()

drill_4['rank'] = drill_4.groupby('year')['global_sales'].rank(method="first", ascending=False)

final =  drill_4[drill_4['rank'] <= 10]




publisher
20th Century Fox Video Games               2
989 Studios                                2
Acclaim Entertainment                      6
Activision                                30
Answer Software                            1
Arena Entertainment                        1
Atari                                     10
Banpresto                                  2
Bethesda Softworks                         3
CPG Products                               1
Capcom                                    12
Coleco                                     1
Disney Interactive Studios                 2
Eidos Interactive                          4
Electronic Arts                           22
Enix Corporation                           6
GT Interactive                             3
HAL Laboratory                             1
Hasbro Interactive                         2
Hudson Soft                                7
Idea Factory                               1
Imagic                                     1


In [15]:
'''DRILL 5 — “Regional mix as percentages (tidy output)”

Create a tidy dataframe:
publisher (top 10 publishers)
region (na/eu/jp/other)
pct_of_publisher_sales

Hints:

first compute sums by publisher for each region
melt to long
compute pct using row-wise division by total'''

drill_5 = (df.groupby('publisher', as_index=False).agg(na = ('na_sales', 'sum'), 
                                        eu = ('eu_sales', 'sum'),
                                        jp = ('jp_sales', 'sum'),
                                        other = ('other_sales', 'sum'),
                                        global_sales = ('global_sales', 'sum'))
                                        .sort_values('global_sales', ascending= False)
                                        .head(10))

drill_5_melt = drill_5.melt(id_vars="publisher", var_name="region", value_name="sales")

drill_5_melt
                                        
                    

,publisher,region,sales
0,Nintendo,na,815.75
1,Electronic Arts,na,584.22
2,Activision,na,426.01
3,Sony Computer Entertainment,na,265.22
4,Ubisoft,na,252.81
5,Take-Two Interactive,na,220.47
6,THQ,na,208.60
7,Konami Digital Entertainment,na,88.91
8,Sega,na,108.78
9,Namco Bandai Games,na,69.38
